In [1]:
# 2023 Gabriel J. Diaz @ RIT

import os
import sys
import numpy as np
import av
import logging
import pickle
from tqdm import tqdm




import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from pathlib import Path, PurePath

logger = logging.getLogger(__name__)
logger.addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
import sys
sys.path.append('../..')

# from retinal_flow_toolkit import .flow_source
from flow_source import *

In [3]:
os.getcwd()
sys.path

['/Users/gjdpci/Documents/GitHub/retinal_flow_toolkit/sandbox/overlay_gaze',
 '/Users/gjdpci/opt/anaconda3/envs/Pytorch_1/lib/python38.zip',
 '/Users/gjdpci/opt/anaconda3/envs/Pytorch_1/lib/python3.8',
 '/Users/gjdpci/opt/anaconda3/envs/Pytorch_1/lib/python3.8/lib-dynload',
 '',
 '/Users/gjdpci/opt/anaconda3/envs/Pytorch_1/lib/python3.8/site-packages',
 '/Users/gjdpci/opt/anaconda3/envs/Pytorch_1/lib/python3.8/site-packages/IPython/extensions',
 '/Users/gjdpci/.ipython',
 '../..',
 'core']

In [4]:
    
# file_path = Path('D:\\Github\\retinal_flow_toolkit\\videos\\Yoyo-LVRA.mp4')

a_file_path = Path(os.path.join('..','..','pupil_labs_data','cb13'))
a_file_path
source = pupil_labs_source(a_file_path)



In [5]:
file_path = source.file_path

In [6]:
def create_containers_and_streams(self, algorithm, visualize_as):

    container_in = av.open(self.file_path)
    average_fps = container_in.streams.video[0].average_rate
    height = container_in.streams.video[0].height
    width = container_in.streams.video[0].width


    ##############################
    # prepare video out
    if visualize_as == 'gaze_overlay':
        video_out_name = self.source_file_name + '_gaze-overlay.mp4' 
    else:
        video_out_name = self.source_file_name + '_' + algorithm + '_' + visualize_as + '.mp4'

    if os.path.isdir(self.video_out_path) is False:
        os.makedirs(self.video_out_path)

    container_out = av.open(os.path.join(self.video_out_path, video_out_name), mode="w", timeout=None)
    
    stream_out = container_out.add_stream("libx264", framerate=average_fps)
    stream_out.options["crf"] = "20"
    stream_out.pix_fmt = container_in.streams.video[0].pix_fmt
    stream_out.time_base = container_in.streams.video[0].time_base
    stream_out = self.set_stream_dimensions(stream_out, visualize_as, height, width)
        
    if visualize_as == 'gaze_overlay':
        overlay_stream = container_out.add_stream("libx264", framerate=average_fps)
        overlay_stream.options["crf"] = "20"
        overlay_stream.pix_fmt = container_in.streams.video[0].pix_fmt #'yuva444p10le'
        overlay_stream.time_base = container_in.streams.video[0].time_base
        overlay_stream = self.set_stream_dimensions(stream_out, visualize_as, height, width)

    print(f'Streams: {len(container_out.streams)}')
    return container_in, container_out

In [7]:
def draw_gaze_overlay(self, frame_width, frame_height, frame_index):
    
    if not type(source.gaze_data ) == pd.DataFrame:
        self.import_gaze_from_exports()
        self.process_gaze_data()
    
    med_xy = self.get_median_gaze_for_frame(frame_index)

    frame = np.zeros((frame_height,frame_width,3), np.uint8)
    
    if med_xy:

        median_x, median_y = med_xy

        height = np.shape(frame)[0]
        width = np.shape(frame)[1]

        frame = cv2.line(frame, (int(width * median_x), 0), (int(width * median_x), height),
                         (255, 0, 0), thickness=2)

        frame = cv2.line(frame, (0, int(height * median_y)), (width, int(height * median_y)),
                         (255, 0, 0), thickness=2)

        cv2.imwrite('temp.png', frame, [cv2.IMWRITE_PNG_COMPRESSION, 0])

        return frame

In [8]:
def import_gaze_from_exports(self):
    gaze_positions_path = os.path.join(self.export_folder, 'gaze_positions.csv')

    if os.path.exists(gaze_positions_path) is False:
        logger.error('No gaze_positions found in the exports folder.')

    # Defaults to the most recent pupil export folder (highest number)
    self.gaze_data = pd.read_csv(gaze_positions_path)

    return True

In [9]:
def overlay_gaze_on_video(self):
    
    container_in, container_out = self.create_containers_and_streams(algorithm=False, 
                                                                                visualize_as='gaze_overlay')
    
    stream_in = container_in.streams.video[0]
    
    num_frames = stream_in.frames
    
    # for raw_frame in container_in.decode(video=0):
    for raw_frame in tqdm(container_in.decode(video=0), desc="Working.", unit= 'frames',total = num_frames):
        
        current_bgr = raw_frame.to_ndarray(format='bgr24')
        current_bgr_processed = self.preprocess_frame(current_bgr)
        
    
        overlay_frame = self.draw_gaze_overlay(raw_frame.width, raw_frame.height, raw_frame.index)
        
#         print('index: ' + str(raw_frame.index))
#         print('overlay: ' + str(raw_frame.index))
        print(container_out.streams)
        
        self.encode_frame(container_out, container_out.streams.video[0], image_out, raw_frame, stream_in)
        self.encode_frame(container_out, container_out.streams.video[1], overlay_frame, raw_frame, stream_in)
    
    self.encode_frame(container_out, container_out.streams.video[0], image_out, raw_frame, stream_in)
    self.encode_frame(container_out, container_out.streams.video[1], overlay_frame, raw_frame, stream_in)

    container_out.close()
    container_in.close()

In [10]:
import types


source.overlay_gaze_on_video = types.MethodType( overlay_gaze_on_video, source )

source.draw_gaze_overlay = types.MethodType( draw_gaze_overlay, source )

source.create_containers_and_streams = types.MethodType( create_containers_and_streams, source )

source.import_gaze_from_exports = types.MethodType( import_gaze_from_exports, source )



In [11]:
source.overlay_gaze_on_video()

Streams: 2


Working.:   0%|                                   | 0/92882 [00:02<?, ?frames/s]


KeyboardInterrupt: 

In [ ]:
# a, b = source.create_containers_and_streams(False, 'gaze_overlay'
                                    )

In [ ]:
# def create_containers_and_streams(self, algorithm, visualize_as):

algorithm = False
visualize_as = 'gaze_overlay'

container_in = av.open(source.file_path)
average_fps = container_in.streams.video[0].average_rate
height = container_in.streams.video[0].height
width = container_in.streams.video[0].width

##############################
# prepare video out
if visualize_as == 'gaze_overlay':
    video_out_name = source.source_file_name + '_gaze-overlay.mp4' 
else:
    video_out_name = source.source_file_name + '_' + algorithm + '_' + visualize_as + '.mp4'

if os.path.isdir(source.video_out_path) is False:
    os.makedirs(source.video_out_path)

container_out = av.open(os.path.join(source.video_out_path, video_out_name), mode="w", timeout=None)

stream_out = container_out.add_stream("libx264", framerate=average_fps)
stream_out.options["crf"] = "20"
stream_out.pix_fmt = container_in.streams.video[0].pix_fmt
stream_out.time_base = container_in.streams.video[0].time_base
stream_out = source.set_stream_dimensions(stream_out, visualize_as, height, width)

# if visualize_as == 'gaze_overlay':
#     overlay_stream = container_out.add_stream("libx264", framerate=average_fps)
#     overlay_stream.options["crf"] = "20"
#     overlay_stream.pix_fmt = container_in.streams.video[0].pix_fmt #'yuva444p10le'
#     overlay_stream.time_base = container_in.streams.video[0].time_base
#     overlay_stream = source.set_stream_dimensions(stream_out, visualize_as, height, width)

print(f'Streams: {len(container_out.streams)}')
# return container_in, container_out

In [ ]:
# container_out.streams.video
stream_out